In [11]:
from langchain.chains import LLMChain, SequentialChain, TransformChain
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
import os


In [2]:
# Define the model to use
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the model to use
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

C:\Users\jim\AppData\Local\Temp\ipykernel_12528\3242483828.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)


In [3]:
email = """I want information on getting a compost bin. I have submitted case number: 9578014."""

# Define the individual chains that make up each step of the complex workflow

### Define the sentiment_chain

In [13]:
# Define a sentiment prompt that returns the given email and a sentiment response
sentiment_prompt = PromptTemplate(
    input_variables=["email"],
    output_key="sentiment",
    template="""
    Given the following email text:
    {email}

    Please provide a sentiment analysis with a response value of 'Negative', 'Neutral', or 'Positive'.

    Format:
    <your sentiment here>
    """
)
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt, output_key="sentiment")

In [14]:
# Test the sentiment chain
result_1 = sentiment_chain.run(email=email)
result_1

C:\Users\jim\AppData\Local\Temp\ipykernel_12528\1363565193.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result_1 = sentiment_chain.run(email=email)


'Neutral'

### Define the topic_chain

In [15]:
# Define the topics and their descriptions
topics_dict = {
    'Homeless': 'words similar to homeless, shelter, encampment',
    'Graffiti': 'words similar to graffiti, paint, tagging',
    'Pothole': 'words similar to pothole, holes',
    'Animal': 'words similar to animal, pest, pets, barking',
    'Vegitation': 'words similar to weeds, trees, limbs, overgrown',
    'Neighborhood': 'words similar to HOA, RNO, sidewalk, fence',
    'Snow Removal': 'words similar to snow, ice, plows',
    'Vehicle': 'words similar to vehicle, car, motorcycle, automobile',
    'Parking': 'words similar to parking',
    'Police': 'words similar to police, gang, loud, drugs, crime',
    'Fireworks': 'words similar to fireworks',
    'Dumping': 'words similar to dumping',
    'Trash': 'words similar to trash, garbage, compost, recycling',
    'Housing': 'words similar to rent, rental, apartments, housing',
    'Policy': 'words similar to policy, tax, taxes, mayor, council, councilwoman, councilman, environmental, environment, rezoning, rezone, government, politics',
    'Street Racing': 'words similar to racing',
    'Transit': 'words similar to transit, traffic, pedestrian, intersection, bicycle, bike, speed, pavement',
    'Parks': 'words similar to park, playground, trails, pool, gym, medians',
}

# Convert the dictionary to a string
topic_descriptions = "\n".join([f"{key}: {desc}" for key, desc in topics_dict.items()])

In [17]:
# Define the prompt template
topic_prompt = PromptTemplate(
    input_variables=["email", "sentiment"],
    output_key="topics",
    template=f"""
    You are an email classification assistant. Based on the content of the email, please classify it into one or more of the following topics:

    {topic_descriptions}

    Email content:
    {{email}}

    Please list the relevant topics based on the email content. If multiple topics apply, separate them with commas. Only return the topic names.
    If you do not know the topic, return 'Other'.

    Format:
    <your topics here>
    """
)
topic_chain = LLMChain(llm=llm, prompt=topic_prompt, output_key="topics")

In [18]:
# Test the sentiment and topic chains
sequential_chain = SequentialChain(
    chains=[sentiment_chain, topic_chain],
    input_variables=["email"],  # Initial input needed
    output_variables=["email", "sentiment", "topics"]  # Final output of the chain
)
result_2 = sequential_chain({"email": email})
print(result_2)


C:\Users\jim\AppData\Local\Temp\ipykernel_12528\3938083916.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result_2 = sequential_chain({"email": email})


{'email': 'I want information on getting a compost bin. I have submitted case number: 9578014.', 'sentiment': 'Neutral', 'topics': 'Trash'}


### Define the rag_retrieval_chain

### Define the rag_chain

In [25]:
# First define the retriever_chain

# Specify the embeddings and directory where the vector store database is located
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
persist_directory = "../chroma_db"
vector_store = Chroma(embedding_function=embedding_function, persist_directory=persist_directory)
retriever = vector_store.as_retriever()

# Step 1: Define the retrieval step
def retrieve_documents(email):
    docs = retriever.invoke(email)
    # Join the content of documents to pass as input for the next step
    return " ".join([doc.page_content for doc in docs])

In [26]:
retrieve_documents(email)

"affected_address: 8101 E Dartmouth Ave Unit 9\ncase_number: 0\ndate: 2024-09-17\nconstituent_email_1: We are hoping you can help us. When we were informed that our trash/recycle/compost barrels would be charged by the size of our trash barrel, we carefully considered the size ordered based on an assumption our trash needs would decrease because we would discard compost in a separate barrel. Unsurprisingly, we find the smaller trash barrel inadequate because we have yet to receive a compost barrel. We have learned that we will not be line for a compost barrel until the end of 2025. The small credit to our quarterly bill hardly compensates for the issues this causes us to discard all trash and compost each week.  Is there any way you can help get us a compost barrel sooner? We would be happy to pick it up at the barrel distribution site.\nd4_response_1: Good afternoon Jeff, \n\nThank you for reaching out to our office and I sincerely apologize for the late response. You are accurate abo

In [27]:
retrieval_chain = TransformChain(input_variables=["email"], output_variables=["context"], transform=retrieve_documents, output_key="context")


In [31]:
result_4 = retrieval_chain({"email": email})
result_4

TypeError: expected string or buffer

In [28]:
# Define the rag_response_chain

TEMPLATE_TEXT_D4_SPECIFIC = """
You are an assistant for question-answering tasks specifically for generating responses to constituent emails.
You work as a senior aide to Councilwoman Diana Romero Campbell, a Denver City Council member for District 4.
You represent the South East Region of the city and county of Denver Colorado USA.
Use the following pieces of retrieved context to help answer the question and generate a response to the constituent.
If you don't know the answer, just say that you don't know but we will get the information and get back to you.
Use three to four sentences maximum, keep the answer concise, and be specific to the city and county of Denver.
In your response, please include a fun fact about the city of Denver.

Question: {email}
Context: {context}
Answer:
"""

rag_response_prompt = PromptTemplate(
    input_variables=["email", "context"],
    output_key="rag_response",
    template=TEMPLATE_TEXT_D4_SPECIFIC
)
rag_response_chain = LLMChain(llm=llm, prompt=rag_response_prompt, output_key="rag_response")

In [29]:
sequential_chain_3 = SequentialChain(
    chains=[sentiment_chain, topic_chain, retrieval_chain, rag_response_chain],
    input_variables=["email"],  # Initial input needed
    output_variables=["email", "sentiment", "topics", "rag_response"]  # Final output of the chain
)
result_3 = sequential_chain_3({"email": email})
print(result_2)


TypeError: expected string or buffer

In [ ]:

# Use a standard propmt template to perform simple queries on the loaded vectorstore
prompt = hub.pull("rlm/rag-prompt")
# [HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

# Instantiate the LLM to use
llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

# Define the chain
rag_chain = (
    {"context": retriever | format_docs, 
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(prompt)

### Define the general_response_chain to search the denvergov.org/Government website

### Define the specific_311_topic_chain

### Define the fun_fact_chain